Analyser les ventes d'une librairie avec R ou Python

# On importe les librairies utiles au projet

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns

# On importe les fichiers CSV dans lesquels se trouvent nos bases de données

products = pd.read_csv("C:/Livrables/DAN-P6-donnees/products.csv")
customers = pd.read_csv("C:/Livrables/DAN-P6-donnees/customers.csv")
transactions = pd.read_csv("C:/Livrables/DAN-P6-donnees/transactions.csv")

products.head()

customers.head()

transactions.head()

products.info()

customers.info()

transactions.info()

# 1. Constitution de notre de base de données complète

## 1.1 Nettoyage de données

transactions = transactions[transactions.session_id	 != "s_0"]

transactions.info()

## 1.2 Merging


# Je modifie mon merge et le met en "right join" afin de ne pas exclure des valeurs manquantes. 
global_ = pd.merge(products, transactions, how="right", on="id_prod")

global_.info()

#Je sauvegarde les données dont le prix et la catégorie manquent dans une variable.
transactions_null = global_[global_["price"].isnull()]

global_final = pd.merge(global_, customers, on="client_id")

## 1.3 Nettoyage des données après merge 

global_final['date'] = pd.to_datetime(global_final['date'])

global_final[global_final['date'].dt.month != 10] 

global_final.info()

global_final.set_index('date', inplace=True)
global_final.info()

global_final = global_final.drop(global_final.index[global_final.index.year.isin([2021])] & global_final.index[global_final.index.month.isin([10])])

global_final = global_final.reset_index()

# 2. Extraction de données et visualisation graphique

## 2.1 Courbe de l'évolution du chiffre d'affaire

chiffre_daffaire = global_final.groupby([pd.Grouper(key ='date', freq='W')]).price.sum()

chiffre_daffaire

MA = chiffre_daffaire.rolling(window=4, min_periods=1).mean()

MA.plot()

plt.figure(figsize=(12,8))
chiffre_daffaire.plot(label='CA')
MA.plot(label='MA')
plt.title("Chiffre d'affaire et MA9")
plt.ylabel('CA')
plt.legend()
plt.grid(True)
plt.show()

## 2.2 Statistiques sur le chiffre d'affaire 

#### 2.2.1 Chiffre d'affaire au total

CA_total = global_final["price"].sum()

CA_total

# Je note que des outliers doivent être supprimés.

rank_client = global_final.groupby("client_id").price.sum().reset_index()

rank_client_sorted = rank_client.sort_values(by="price",ascending=False).head(10)

sns.set_theme(style="whitegrid")


f, ax = plt.subplots(figsize=(10, 10))


sns.set_color_codes("pastel")
sns.barplot(x="client_id", y="price", data=rank_client_sorted,
            label="Total", color="b")


ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(xlim=(0, 10), ylabel="CA",
       xlabel="ID du client")
sns.despine(left=True, bottom=True)

# J'exclue les données des clients-outliers après mon CA global et avant mes opérations sur les données. 

global_final = global_final[global_final['client_id'] != "c_1609"] 
global_final = global_final[global_final['client_id'] != "c_4958"]
global_final = global_final[global_final['client_id'] != "c_6714"]
global_final = global_final[global_final['client_id'] != "c_3454"]

#Je vérifie que mes données ont bien été supprimées.

global_final.info()

#### 2.2.2 Répartition du chiffre d'affaire selon prix

CA_10 = global_final[global_final["price"] <10]

print("Le chiffre d'affaire des livres de moins de 10€ est de:",round(CA_10.price.sum()/CA_total*100,2), "%")

CA_10_25 = global_final[global_final["price"].between(10, 25)]

print("Le chiffre d'affaire des livres entre 10€ et 25€ est de:",round(CA_10_25.price.sum()/CA_total*100,2), "%")

CA_25_100 = global_final[global_final["price"].between(25, 100)]

print("Le chiffre d'affaire des livres entre 25€ et 100€ est de:",round(CA_25_100.price.sum()/CA_total*100,2), "%")

CA_100 = global_final[global_final["price"] >100]

print("Le chiffre d'affaire des livres de plus de 100€ est de:",round(CA_100.price.sum()/CA_total*100,2), "%")

global_final.groupby("id_prod")

bins = [-1, 10, 25, 100, np.inf]
labels=['0-10', '10-25', '25-100', 'more than 100']
df=global_final.groupby(pd.cut(global_final['price'], bins=bins, labels=labels)).size().reset_index(name="ventes")
print (df)

bins = [-1, 10, 25, 100, np.inf]
labels=['0-10', '10-25', '25-100', 'more than 100']
df=global_final.groupby(pd.cut(global_final['price'], bins=bins, labels=labels)).sum()/CA_total*100
print (df)

bins = [-1, 10, 25, 100, np.inf]
labels=['0-10', '10-25', '25-100', 'more than 100']
df=global_final.groupby(pd.cut(global_final['price'], bins=bins, labels=labels)).size().reset_index(name="ventes")
print (df)

plt.pie(df["ventes"], labels=df.ventes, radius=2, center=(4, 4))

bins = [-1, 10, 25, 100, np.inf]
labels=['0-10', '10-25', '25-100', 'more than 100']
dfg=round(global_final.groupby(pd.cut(global_final['price'], bins=bins, labels=labels)).sum()/CA_total*100,2)
print (dfg)

plt.pie(dfg.price, labels=dfg.price, radius=2, center=(4, 4))

#### 2.2.3 Visualisation du chiffre d'affaire par produit 

ca_par_produit = global_final.groupby("id_prod").price.sum().reset_index()

In [49]:
ca_par_produit

,id_prod,price
0,0_0,4128.75
1,0_1,4934.51
2,0_10,287.20
3,0_100,61.80
4,0_1000,2558.16
...,...,...
3253,2_95,395.96
3254,2_96,25679.76
3255,2_97,1770.89
3256,2_98,149.74


plt.figure()

df = pd.DataFrame(ca_par_produit,columns=['id_prod','price'])
df.plot(x ='id_prod', y='price', label='CA', c="green")
plt.title("Vue d'ensemble du CA par produit")
plt.ylabel('CA')
plt.legend()
plt.grid(True)
plt.show()

In [51]:
ca_par_produit_sorted = ca_par_produit.sort_values(by="price", ascending=False).head(10)

ca_par_produit_sorted_2 = ca_par_produit.sort_values(by="price").head(10)

ca_par_produit_sorted_2

## 2.3 Courbe de Lorenz du chiffre d'affaire par produit

plt.figure()

df = pd.DataFrame(ca_par_produit_sorted,columns=['id_prod','price'])
df.plot(x ='id_prod', y='price', label='CA', c="green")
plt.title("Vue d'ensemble du CA par produit")
plt.ylabel('CA')
plt.legend()
plt.grid(True)
plt.show()

## 2.4 Produits les mieux vendus

Classification des produits qui se vendent le mieux et le moins bien

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")


f, ax = plt.subplots(figsize=(10,10))


sns.set_color_codes("pastel")
sns.barplot(x="id_prod", y="price", data=ca_par_produit_sorted,
            label="Total", color="b")


ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(xlim=(0, 10), ylabel="CA",
       xlabel="ID du produit")
sns.despine(left=True, bottom=True)

## 2.5 Produits les moins vendus

sns.set_theme(style="whitegrid")


f, ax = plt.subplots(figsize=(10, 10))


sns.set_color_codes("pastel")
sns.barplot(x="id_prod", y="price", data=ca_par_produit_sorted_2,
            label="Total", color="b")


ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(xlim=(0, 10), ylabel="CA",
       xlabel="ID du produit")
sns.despine(left=True, bottom=True)

# 3. Liens entre différentes variables (graphiques)

## 3.1 Lien entre genre et catégorie de livre achetée

genre = global_final.groupby(["sex"]).count()

plt.pie(genre.id_prod,radius=1, center=(4, 4),labels = ["F", "M"], colors = ['pink', "yellow"], autopct = lambda genre: str(round(genre, 1)) + '%')
plt.title("Répartition des ventes par sexe")

ages = global_final["birth"].sort_values(ascending = True)

ca_clients = global_final.groupby("client_id").price.sum()

categ_femme = global_final.loc[global_final['sex'] == "f",:].groupby("categ").size()

categ_homme = global_final.loc[global_final['sex'] == "m",:].groupby("categ").size()

categ_femme.reset_index()

categ_homme.reset_index()

plt.pie(categ_homme,radius=1, center=(4, 4), labels = ["1", "2", "3"], colors = ['orange', "yellow", "white"], autopct = lambda categ_homme: str(round(categ_homme, 1)) + '%')
plt.title("Répartition des ventes par catégorie pour les hommes")

plt.pie(categ_femme, radius=1, center=(4, 4), labels = ["1", "2", "3"], colors = ['pink', "yellow", "white"], autopct = lambda categ_femme: str(round(categ_femme, 1)) + '%')
plt.title("Répartition des ventes par catégorie pour les femmes")

## 3.2 Lien entre âge et chiffre d'affaire

age_total = global_final.groupby(["birth"]).sum()

plt.figure(figsize=(12,8))
age_total["price"].plot()
plt.xlabel('Year of birth')
plt.ylabel('CA')
plt.title("Chiffre d'affaire selon l'année de naissance du client")
plt.grid(True)

## 3.2 Lien entre âge et fréquence d'achat

age_frequence = global_final.groupby(["birth"]).size()

plt.figure(figsize=(12,8))
age_frequence.plot()
plt.xlabel('Year of birth')
plt.ylabel('CA')
plt.title("Fréquence d'achat selon l'année de naissance du client")
plt.grid(True)

## 3.3 Lien entre âge et panier moyen

age_mean = global_final.groupby(["birth"]).mean()

plt.figure(figsize=(12,8))
age_mean["price"].plot()
plt.xlabel('Year of birth')
plt.ylabel('Prix')
plt.title("Panier moyen selon l'âge moyen du client")
plt.grid(True)


## 3.4 Lien entre âge et catégorie de livre acheté

plt.figure(figsize=(12,8))

age_mean["categ"].plot()
plt.xlabel('Year of birth')
plt.ylabel('Catégorie')

plt.title("Catégorie préférée selon l'année de naissance du client")
plt.grid(True)


categ_plot = global_final.groupby("client_id").sum()

categ_plot.head()

categ_0 = global_final.loc[global_final["categ"]==0]

categ_1 = global_final.loc[global_final["categ"]==1]

categ_2 = global_final.loc[global_final["categ"]==2]

categ_0_stacked = categ_0.groupby([pd.Grouper(key ='date', freq='M')]).price.sum()

categ_0_stacked.plot()

categ_1_stacked = categ_1.groupby([pd.Grouper(key ='date', freq='M')]).price.sum()

categ_2_stacked = categ_2.groupby([pd.Grouper(key ='date', freq='M')]).price.sum()

categ_2_stacked.plot()

plt.figure(figsize=(12,8))
plt.plot(categ_0_stacked)
plt.plot(categ_1_stacked)
plt.plot(categ_2_stacked)

plt.xlabel('Date')
plt.ylabel("Chiffre d'affaire")

plt.title("Evolution du chiffre d'affaire par catégorie")
plt.grid(True)


categ_stacked = global_final.groupby([pd.Grouper(key ='date', freq='M')]).price.sum()

categ_stacked.plot()

age_1 = global_final[global_final["birth"] <1970]

age_1 = age_1.groupby('categ').size()

age_1.plot(kind='bar')

age_2 = global_final[global_final["birth"].between(1970, 1990)]

age_2 = age_2.groupby('categ').size()

age_2.plot(kind='bar')

age_3 = global_final[global_final["birth"] >1990]

age_3 = age_3.groupby('categ').size()

age_3.plot(kind='bar')

# 4. Liens entre différentes variables (stats)

## 4.1 Corrélation entre âge et genre 

client1 = global_final.groupby("client_id").head(10)

client1

X = global_final["sex"]
Y = global_final["categ"]
data = {
    'sex':X,
    'categ':Y
}
df = pd.DataFrame(data)

df

X = "sex"
Y = "categ"

cont = df[[X, Y]].pivot_table(index=X, columns=Y, aggfunc=len).fillna(0).copy()

tx = df[X].value_counts()
ty = df[Y].value_counts()

cont = cont.astype(int)

cont

st_chi2, st_p, st_dof, st_exp = st.chi2_contingency(cont)

st_p

## 4.2 Corrélation entre âge et prix 

test_kama_mean = global_final.groupby(['birth']).price.mean().reset_index()

from scipy.stats import pearsonr

pearson, _ = pearsonr(test_kama_mean.birth, test_kama_mean.price)

pearson

# Il y a une corrélation positive entre les deux variables, mais elle n'est pas totale